<h1 style = "font-size:3rem;color:darkcyan"> Generate Audio </h1>
& compare original audio with the reconstructed audio

In [1]:
# import libraries
import numpy as np
import json
import librosa
import os
import soundfile as sf
import pickle
import IPython

In [2]:
from autoencoder import VAE

# Sound gen class

In [3]:
class SoundGenerator:
    
    def __init__(self, vae, hop_length):
        self.vae = vae
        self.hop_length = hop_length
        self.min = 0.
        self.max = 1.
        
    def generate(self, spectrograms, min_max_values):
        generated_spectrograms, latent_representations = self.vae.reconstruct(spectrograms)
        signals = self.convert_spectrograms_to_audio(generated_spectrograms, min_max_values)
        return signals, latent_representations
    
    def convert_spectrograms_to_audio(self, spectrograms, min_max_values):
        signals = []
        for spectrogram, min_max_value in zip(spectrograms, min_max_values):
            # reshape: get rid of third dim
            log_spectrogram = spectrogram[:, :, 0]
    
            # apply denormalization
            denorm_log_spectrogram = self._denormalize(log_spectrogram,
                                                       min_max_value[0],
                                                       min_max_value[1])
            # log to linear spectrogram
            spec = librosa.db_to_amplitude(denorm_log_spectrogram)
            
            # apply STFT using Griffin Lim
            signal = librosa.istft(spec, hop_length = self.hop_length)
            
            # append signal to 'signals' array
            signals.append(signal)
            
        return signals
        
    def _denormalize(self, norm_signal, original_min, original_max):
        signal = (norm_signal - self.min) / (self.max - self.min)
        signal = signal * (original_max - original_min) + original_min
        return signal
        

# Initialize sound generator

In [4]:
vae = VAE.load('model')
vae.summary()
hop_length = 256
sound_generator = SoundGenerator(vae, hop_length)


# sample spectrograms + min_max_values
# generate audio for sampled spectrograms
# convert spectrograms to audio (for comparison)
# save audio signal

Instructions for updating:
Colocations handled automatically by placer.
Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_input (InputLayer)     [(None, 256, 64, 1)  0           []                               
                                ]                                                                 
                                                                                                  
 encoder_conv_layer_1 (Conv2D)  (None, 128, 32, 512  5120        ['encoder_input[0][0]']          
                                )                                                                 
                                                                                                  
 encoder_relu_1 (ReLU)          (None, 128, 32, 512  0           ['encoder_conv_layer_1[0][0]']   
                    

 encoder (Functional)        (None, 128)               1879008   
                                                                 
 decoder (Functional)        (None, 256, 64, 1)        443361    
                                                                 
Total params: 2,322,369
Trainable params: 2,319,937
Non-trainable params: 2,432
_________________________________________________________________


# Load spectrograms + min_max_values

In [5]:
def import_dataset(dataset_path):
    with open(dataset_path, 'r') as f:
        data = json.load(f)
    
    # split list into different np arrays
    inputs = np.array(data['log_spectrogram'])
    filenames = np.array(data['filenames'])
    min_max_values = np.array(data['min_max_values'])
    
    # reshape to add one dimension to features for CNN
    inputs = inputs[..., np.newaxis] 
    return inputs, filenames, min_max_values

spectrograms, filenames, min_max_values = import_dataset('data.json')

# Sample spectrograms

In [6]:
def select_spectrograms(spectrograms,
                        filenames,
                        min_max_values,
                        num_spectrograms=2):
    sampled_indexes = np.random.choice(range(len(spectrograms)), num_spectrograms)
    sampled_spectrograms = spectrograms[sampled_indexes]
    filenames = [filenames[index] for index in sampled_indexes]
    sampled_min_max_values = [min_max_values[index] for index in
                           sampled_indexes]
    print(filenames)
    print(sampled_min_max_values)
    return sampled_spectrograms, sampled_min_max_values

sampled_specs , sampled_min_max_values = select_spectrograms(spectrograms, 
                                                            filenames,
                                                            min_max_values,
                                                            5)

['5_01_17.wav', '3_01_5.wav', '9_01_12.wav', '8_01_18.wav', '1_01_27.wav']
[array([0., 1.]), array([0., 1.]), array([0., 1.]), array([0., 1.]), array([0., 1.])]


# Generate audio for sampled spectrograms

In [7]:
signals, _ = sound_generator.generate(sampled_specs, sampled_min_max_values)

C:\Users\helme\anaconda3\lib\site-packages\keras\engine\training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


# Convert spec to audio for comparison

In [8]:
original_audio = sound_generator.convert_spectrograms_to_audio(sampled_specs, sampled_min_max_values)

# Save signals

In [10]:
def save_signals(signals, save_dir, sample_rate=22050, is_original = False):
    for i, signal in enumerate(signals):
        save_path = os.path.join(save_dir, str(i) + ".wav")
        if is_original:
            sf.write(save_path, signal, sample_rate)
        else:
            sf.write(save_path, signal*100, sample_rate)
        
save_signals(signals, 'samples/generated', is_original =False)
save_signals(original_audio, 'samples/original', is_original = True)

In [16]:
IPython.display.Audio("samples/original/1.wav")

In [17]:
IPython.display.Audio("samples/generated/1.wav")